# Tutorial for Downloading Omni6DPose Dataset
Omni6DPose is a dataset for universal 6D object pose estimation and tracking, featured by its diversity in object categories, large scale, and variety in object materials. The project is available at [Omni6DPose](https://jiyao06.github.io/Omni6DPose/). In this tutorial, we will show you how to download the dataset.

In [ ]:
%pip install requests
%pip install tqdm

### 1. Set up the download configuration
 We need to set up the download configuration first. `DATA_PATH` is the path to save the dataset. **Set the `PROXY` to your own proxy if you are behind a firewall. If you are not behind a firewall, you can set `PROXY` to `None`.**

In [ ]:
############################## Set the configuration for loading data ###########################
DATA_PATH = 'data/Omni6DPose/'              # Path to the directory where the data will be stored
PROXY = 'http://proxy-server:proxy-port'    # Proxy server to use for downloading the data
#################################################################################################

import json, os, sys, requests, zipfile
from tqdm import tqdm
from time import sleep

# Set the proxy server
proxies = {'http': PROXY, 'https': PROXY}

# Get the download links and create the data directory
with open('data_links.json') as f:
    data = json.load(f)
for key in data:
    os.makedirs(DATA_PATH + key, exist_ok=True)


def check_zip_file_integrity(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_file:
            bad_file = zip_file.testzip()
            if bad_file:
                return False
            else:
                return True
    except zipfile.BadZipFile:
        return False
    except Exception as e:
        return False


def download_with_retries(url, proxies=None, timeout=5, max_retries=5, retry_delay=2):
    """
    Attempts to download content with a retry mechanism on timeout or connection errors.
    
    Parameters:
    - url (str): The URL of the resource to download.
    - proxies (dict): Proxy settings.
    - timeout (int or float): Request timeout in seconds.
    - max_retries (int): Maximum number of retries.
    - retry_delay (int or float): Delay between retries in seconds.
    
    Returns:
    - response: The Response object if successful, None if all retries fail.
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, stream=True, proxies=proxies, timeout=timeout)
            response.raise_for_status()  # Check for HTTP errors
            return response  # Successful response, return the response object
        except (requests.Timeout, requests.ConnectionError):
            print(f"Request timed out or connection error, attempting retry {attempt + 1}...")
            sleep(retry_delay)
        except requests.HTTPError as err:
            print(f"HTTP error: {err}")
            return None  # On HTTP error, return None
    print("All retry attempts failed")
    return None  # All retries failed, return None


def download_file(url, filename):
    progress_bar = None
    try:
        if os.path.exists(filename) :
            if filename.endswith('.zip') and not check_zip_file_integrity(filename):
                print('\033[93m' + f"{filename} already exists but is corrupted! Re-downloading..." + '\033[0m')
            else:
                print(f"{filename} already exists!")
                return True
        else:
            print('\033[93m' + f"Downloading {filename}..." + '\033[0m')
        url = url.replace('dl=0', 'dl=1')
        # response = requests.get(url, stream=True, proxies=proxies)     
        response = download_with_retries(url, proxies=proxies)
        total_size_in_bytes = int(response.headers.get('content-length', 0))
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)   
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    progress_bar.update(len(chunk))
                    f.write(chunk)
            progress_bar.close()
            return True
        else:
            print('\033[91m' + f"Failed to download {filename}! The server returned status code {response.status_code}" + '\033[0m')
            return False
    
    except (KeyboardInterrupt, requests.RequestException, Exception) as e:
        if os.path.exists(filename):
            os.remove(filename)
        print('\033[91m' + f"An error occurred while downloading {filename}! {e} The file has been removed." + '\033[0m')
        return False
    
    finally:
        if progress_bar is not None:
            progress_bar.close()

### 2. Download Meta files
The Meta files are available at [Meta](https://www.dropbox.com/scl/fo/xumdcn3yp4hi0ahrg0v23/AMEANjZJoDpg5eeccQ02KnQ?rlkey=csw09o0uoh2q6ia93wcpp3mea&st=sru933q5&dl=0). You can download the files using the following command. The files is about 4MB.

In [ ]:
Meta_links = data['Meta']
fail_list = []
for item, link in Meta_links.items():
    print('\033[93m' + f"Downloading {item} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'Meta', exist_ok=True)
    res = download_file(link, f"{DATA_PATH}Meta/{item}")
    if not res:
        fail_list.append(item)
if len(fail_list) == 0:
    print('\033[92m' + "Meta data downloaded successfully!" + '\033[0m')
else:
    print('\n' + '-'*100)
    print('\033[91m' + "Meta data download failed for the following files:" + '\033[0m')
    print(fail_list)
    print('\033[91m' + "Please rerun the cell to retry downloading the failed files." + '\033[0m')

### 3. Download PAM dataset [Optional]
The PAM dataset is available at [PAM](https://www.dropbox.com/scl/fo/oxpad3hjijwehrbdp0krf/AHaWA2xhz0lN-x6elq3Sf2I?rlkey=sctuyqczvmqukth4z4lqk2uj7&st=xx2xpwky&dl=0). You can download the dataset using the following command. The dataset is about 14GB. PAM contains the pose aligned 3D models, which is not necessary for the benchmarking if your method does not require the reference 3D models.

In [ ]:
PAM_links = data['PAM']
fail_list = []
for item, link in PAM_links.items():
    print('\033[93m' + f"Downloading {item} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'PAM', exist_ok=True)
    res = download_file(link, f"{DATA_PATH}PAM/{item}")
    if not res:
        fail_list.append(item)
if len(fail_list) == 0:
    print('\033[92m' + "PAM data downloaded successfully!" + '\033[0m')
else:
    print('\n' + '-'*100)
    print('\033[91m' + "PAM data download failed for the following files:" + '\033[0m')
    print(fail_list)
    print('\033[91m' + "Please rerun the cell to retry downloading the failed files." + '\033[0m')

### 4. Download ROPE dataset
The ROPE dataset is available at [ROPE](https://www.dropbox.com/scl/fo/). You can download the dataset using the following command. The dataset contains 363 scenes, with each scene being approximately 500MB. The total size of the dataset is about 180GB. If you want to download a subset of the dataset, you can set the parameter `SCENE_NUM` in the following scripts. If the download is interrupted, you can rerun the script, and it will skip the downloaded files.

In [ ]:
################################### Set the configuration for loading the ROPE data ###################################
FILES = ['meta.zip', 'color.zip', 'depth.zip', 'mask.zip', 'mask_sam.zip']   # The files to be downloaded
SCENE_NUM = 2                                                                # The number of scenes to be downloaded
#######################################################################################################################

assert SCENE_NUM > 0 and SCENE_NUM <= 363, 'Invalid scene number'
ROPE_links = data['ROPE']
scenes = list(ROPE_links.keys())
scene_ids = [scene.split('-')[0] for scene in scenes]
scene_ids = sorted(list(set(scene_ids)))[:SCENE_NUM]
print(f"The selected scenes are: {scene_ids}")

fail_list = []
for index, scene in enumerate(scene_ids):
    print('----------------------------------------------------------------------------------------------------')
    print('\033[93m' + f"[{index+1}/{len(scene_ids)}] Downloading scene {scene} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'ROPE/' + scene, exist_ok=True)
    for file in FILES:
        res = download_file(ROPE_links[f"{scene}-{file}"], f"{DATA_PATH}ROPE/{scene}/{file}")
        if not res:
            fail_list.append(f"{scene}-{file}")
    print('----------------------------------------------------------------------------------------------------')
if len(fail_list) == 0:
    print('\033[92m' + "ROPE data downloaded successfully!" + '\033[0m')
else:
    print('\n' + '-'*100)
    print('\033[91m' + "ROPE data download failed for the following files:" + '\033[0m')
    print(fail_list)
    print('\033[91m' + "Please rerun the cell to retry downloading the failed files." + '\033[0m')

### 5. Download SOPE dataset
The SOPE dataset is available at [SOPE](). You can download the dataset using the following command. The dataset comprises 51 patches. If you want to download a subset of the dataset, you can set the `PATCH_NUM` parameter in the scripts below. If the download is interrupted, you can rerun the script, and it will skip the downloaded files.

The files `ir.zip`, `depth_1.zip`, and `coord.zip` are not necessary for benchmarking. Specifically, `ir.zip` contains the simulated infrared images, `depth_1.zip` includes the perfect depth images, and `coord.zip` contains the NOCS map proposed in [NOCS](). If you wish to download these files, you can set the `DOWNLOAD_IR`, `DOWNLOAD_GT_DEPTH`, and `DOWNLOAD_NOCS` parameters to `True` in the scripts below. Each batch of the dataset without the optional files is about 14GB. The total size of the dataset is about 720GB.

In [ ]:
############################################ Set the configuration for loading the SOPE data ############################################
FILES = ['meta.zip', 'color.zip', 'depth.zip', 'depth_1.zip', 'mask.zip', 'coord.zip', 'ir.zip'] # The selected files to be downloaded
DOWNLOAD_IR = False                                                                              # Download the IR images
DOWNLOAD_GT_DEPTH = False                                                                        # Download the perfect depth images
DOWNLOAD_NOCS = False                                                                            # Download the NOCS map
PATCH_NUM = 2                                                                                    # The number of patches to be downloaded
#########################################################################################################################################

FILES.remove('ir.zip') if not DOWNLOAD_IR else None
FILES.remove('depth_1.zip') if not DOWNLOAD_GT_DEPTH else None
FILES.remove('coord.zip') if not DOWNLOAD_NOCS else None

assert PATCH_NUM > 0 and PATCH_NUM <= 50, 'Invalid patch number'
SOPE_links = data['SOPE']
patches = list(SOPE_links.keys())
patch_ids = [patch.split('-')[0] for patch in patches]
patch_ids = sorted(list(set(patch_ids)))[:PATCH_NUM]
print('\033[93m' + f"The total number of patches is: {len(patch_ids)}" + '\033[0m')
print('\033[93m' +  f'The selected patches are: {patch_ids}' + '\033[0m')

fail_list = []
for index, patch in enumerate(patch_ids):
    print('----------------------------------------------------------------------------------------------------')
    print('\033[93m' + f"[{index+1}/{len(patch_ids)}] Downloading patch {patch} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'SOPE/' + patch + '/train', exist_ok=True)
    os.makedirs(DATA_PATH + 'SOPE/' + patch + '/test', exist_ok=True)
    for file in FILES:
        res_train = download_file(SOPE_links[f"{patch}-{'train'}-{file}"], f"{DATA_PATH}SOPE/{patch}/train/{file}")
        res_test = download_file(SOPE_links[f"{patch}-{'test'}-{file}"], f"{DATA_PATH}SOPE/{patch}/test/{file}")
        if not res_train:
            fail_list.append(f"{patch}-{'train'}-{file}")
        if not res_test:
            fail_list.append(f"{patch}-{'test'}-{file}")
    print('----------------------------------------------------------------------------------------------------')
if len(fail_list) == 0:
    print('\033[92m' + "SOPE data downloaded successfully!" + '\033[0m')
else:
    print('\n' + '-'*100)
    print('\033[91m' + "SOPE data download failed for the following files:" + '\033[0m')
    print(fail_list)
    print('\033[91m' + "Please rerun the cell to retry downloading the failed files." + '\033[0m')

### 6. Unzip the files
After downloading the files, you can unzip the files using the following command. If you want to remove the zip files after unzipping, you can set the `REMOVE_ZIP` parameter to `True` in the scripts below. The files will be unzipped to the `./data` directory, and the directory structure will be as follows:
```
data
└── Omni6DPose
    ├── ROPE
    │   ├── SCENE_ID
    │   │   ├── FRAME_ID_meta.json
    │   │   ├── FRAME_ID_color.png
    │   │   ├── FRAME_ID_mask.exr
    │   │   ├── FRAME_ID_depth.exr
    │   │   ├── FRAME_ID_mask_sam.npz [Optional]
    │   │   └── ...
    │   └── ...
    ├── SOPE
    │   ├── PATCH_ID
    │   │   ├── train
    │   │   │   ├── SCENE_NAME
    │   │   │   |  ├── SCENE_ID
    │   |   |   |  |   ├── FRAME_ID_meta.json
    │   │   │   |  |   ├── FRAME_ID_color.png
    │   │   │   |  |   ├── FRAME_ID_mask.exr
    │   |   |   |  |   ├── FRAME_ID_depth.exr
    │   │   │   |  |   ├── FRAME_ID_depth_1.exr [Optional]
    │   │   │   |  |   ├── FRAME_ID_coord.png   [Optional]
    │   │   │   |  |   ├── FRAME_ID_ir_l.png    [Optional]
    │   │   │   |  |   ├── FRAME_ID_ir_r.png    [Optional]
    │   │   │   |  |   └── ...
    │   │   │   |  └── ...
    │   │   │   └── ...
    │   │   └── test
    │   └── ...
    ├── PAM
    │   └── obj_meshes
    │       ├── DATASET-CLASS_ID
    │       └── ...
    └── Meta    
        ├── obj_meta.json 
        └── real_obj_meta.json
```


In [ ]:
################################### Set the configuration for unzipping the downloaded files ##################################
REMOVE_ZIPS = True   # Remove the zip files after unzipping
UNZIP_PAM = True     # Unzip the PAM files
UNZIP_ROPE = True    # Unzip the ROPE files
UNZIP_SOPE = True    # Unzip the SOPE files
###############################################################################################################################

import zipfile
import os
from tqdm import tqdm

def unzip_files(directory, remove_zips=False):
    files = os.listdir(directory)
    for file in files:
        if file.endswith('.zip'):
            with zipfile.ZipFile(directory + file, 'r') as zip_ref:
                zip_ref.extractall(directory)
            if remove_zips:
                os.remove(directory + file)

# Unzip the PAM files
if UNZIP_PAM:
    print('\033[93m' + f"Unzipping files in {DATA_PATH + 'PAM/'} ..." + '\033[0m')
    unzip_files(DATA_PATH + 'PAM/', remove_zips=REMOVE_ZIPS)

# Unzip the ROPE files
if UNZIP_ROPE:
    scenes = os.listdir(DATA_PATH + 'ROPE/')
    print('\033[93m' + f"Unzipping files in {DATA_PATH + 'ROPE/'} ..." + '\033[0m')
    for scene in tqdm(scenes):
        unzip_files(DATA_PATH + 'ROPE/' + scene + '/', remove_zips=REMOVE_ZIPS)
    
# Unzip the SOPE files
if UNZIP_SOPE:
    patches = os.listdir(DATA_PATH + 'SOPE/')
    print('\033[93m' + f"Unzipping files in {DATA_PATH + 'SOPE/'} ..." + '\033[0m')
    for patch in tqdm(patches):
        print('\033[93m' + f"Unzipping files in {DATA_PATH + 'SOPE/' + patch} ..." + '\033[0m')
        unzip_files(DATA_PATH + 'SOPE/' + patch + '/train/', remove_zips=REMOVE_ZIPS)
        unzip_files(DATA_PATH + 'SOPE/' + patch + '/test/', remove_zips=REMOVE_ZIPS)
    print('\033[92m' + "All files unzipped successfully!" + '\033[0m')